In [1]:
import pandas as pd
import sklearn.preprocessing as pre
import xgboost as xgb
#pip install xgboost
import matplotlib.pyplot as plt
import seaborn as sns
#pip install sns
import numpy as np

/home/zzq/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_df = pd.read_csv("/home/zzq/kaggle/house/train.csv")

In [108]:
test_df = pd.read_csv("/home/zzq/kaggle/house/test.csv")
test_miss=test_df.isnull().sum(axis=0).reset_index()
test_miss.columns=['name','count']
test_miss=test_miss.ix[test_miss['count']>0]
miss_name=test_miss['name'].values
print miss_name.shape
test_name=['id','timestamp']
for i in range(48):
    test_name.append(str(miss_name[i]))

(48,)


In [3]:
#将有空值的提取出来
missing_df = train_df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.ix[missing_df['missing_count']>0]

In [31]:
print type(missing_df)

<class 'pandas.core.frame.DataFrame'>


In [112]:
# print missing_df
name=missing_df['column_name'].values
name_list=[]
print name.shape
for i in range(51):
    name_list.append(str(name[i]))

(51,)


In [113]:
name_list.append('id')
name_list.append('timestamp')
print len(name_list)

53


In [136]:
test_feature=test_df.drop(name_list,axis=1)
test_feature_nd=test_feature.as_matrix()
print test_feature_nd.shape
print type(test_feature_nd[1][1])
if type(test_feature_nd[1][1])==type('str'):
    print 'str'

(7662, 238)
<type 'str'>
str


In [138]:
import types
max_=test_feature_nd.max(axis=0)
min_=test_feature_nd.min(axis=0)
print max_
for i in range(238):
    if type(max_[i])==type('demo'):
        max_[i]=1.0
    if type(min_[i])==type('demo'):
        min_[i]=0.0
    if max_[i]==min_[i]:
        max_[i]=1.0
        
for i in range(7662):
    for j in range(238):
        if type(test_feature_nd[i][j])==type('Occ'):
            test_feature_nd[i][j]=0.0
        else:
            if test_feature_nd[i][j]==max_[j]:
                test_feature_nd[i][j]=1.0
            else:
                test_feature_nd[i][j]=(float(test_feature_nd[i][j])-float(min_[j]))/(max_[j]-min_[j])

[1.0 0.0 0.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 0.0 1.0
 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
 0.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 0.0 1.0 1.0 1.0 0.0 1.0 1.0 1.0 1.0 1.0
 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 0.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
 1.0 1.0 1.0 1.0]


In [139]:
with open('/home/zzq/kaggle/house/test_file/feature_238.csv','w') as test:
    np.savetxt(test,test_feature_nd,delimiter=',')

In [43]:
name_list.append('id')
name_list.append('timestamp')
name_list.append('price_doc')
feature=train_df.drop(name_list, axis=1)
print type(feature)

<class 'pandas.core.frame.DataFrame'>


In [48]:
print feature.index
print feature.shape
feature_nd=feature.as_matrix()

RangeIndex(start=0, stop=30471, step=1)


(30471, 238)


In [50]:
print type(feature_nd)
print feature_nd.shape

<type 'numpy.ndarray'>
(30471, 238)


In [32]:
#价钱值
#将奇值去掉
ulimit = np.percentile(train_df.price_doc.values, 99.5)
llimit = np.percentile(train_df.price_doc.values, 0.5)
train_df['price_doc'].ix[train_df['price_doc']>ulimit] = ulimit
train_df['price_doc'].ix[train_df['price_doc']<llimit] = llimit
train_y = train_df['price_doc'].values

In [34]:
print train_y.shape
print type(train_y)

(30471,)
<type 'numpy.ndarray'>


In [37]:
b=np.reshape(train_y,30471)
print b.shape

(30471,)


In [38]:
for i in range(4):
    print b[i]

5850000.0
6000000.0
5700000.0
13100000.0


In [39]:
price_label=[]
for i in range(30471):
    price_label.append(int(b[i]/1000000))

In [35]:
print train_y

[  5850000.   6000000.   5700000. ...,   6970959.  13500000.   5600000.]


In [36]:
print train_y[0]

5850000.0


In [27]:
# price=np.asarray(train_y)

In [30]:
# print np.shape(price)
# print price[1][0]

()


IndexError: too many indices for array

In [10]:
print train_y.max()
print train_y.min()

32105366.6
990000.0


In [22]:
price=[]
price=train_y
print type(price)
print price.shape

<type 'int'>


AttributeError: 'int' object has no attribute 'shape'

In [17]:
# train_y.tolist()

AttributeError: 'int' object has no attribute 'tolist'

In [16]:
print type(train_y)
train_y_list=list(train_y)
print type(train_y_list)

<type 'int'>


TypeError: 'int' object is not iterable

In [14]:
label=[]
for i in range(30471):
    label.append(train_y[i]/1000000)

TypeError: 'int' object has no attribute '__getitem__'

In [51]:
#what I need is feature_nd and price_label
import tensorflow as tf
def add_layer(input,input_size,output_size,action_fun=None):
    weights=tf.Variable(tf.random_normal([input_size,output_size]))
    biases=tf.Variable(tf.zeros([1,output_size])+0.1)
    ans=tf.matmul(input,weights)+biases
    if action_fun is None:
        output=ans
    else:
        output=action_fun(ans)
    return output

In [52]:
xs=tf.placeholder(tf.float32,[None,1])
ys=tf.placeholder(tf.float32,[None,1])

In [53]:
print feature_nd.shape

(30471, 238)


In [58]:
hidden_layer1=add_layer(xs,1,1000,action_fun=tf.nn.relu)#there must be 1
hidden_layer2=add_layer(hidden_layer1,1000,1000,action_fun=tf.nn.relu)
hidden_layer3=add_layer(hidden_layer2,1000,200,action_fun=tf.nn.relu)
prediction=add_layer(hidden_layer3,200,1,action_fun=None)
loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), reduction_indices = [1]))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [89]:
import types
max_=feature_nd.max(axis=0)
min_=feature_nd.min(axis=0)
for i in range(30471):
    for j in range(238):
        if type(feature_nd[i][j])==type('str'):
            feature_nd[i][j]=0.0
        else:
            if feature_nd[i][j]==max_[j]:
                feature_nd[i][j]=1.0
            else:
                feature_nd[i][j]=(float(feature_nd[i][j])-float(min_[j]))/(float(max_[j])-float(min_[j]))

In [91]:
import os
if os.path.isfile('/home/zzq/kaggle/house/feartures/feature_238.csv'):
    os.remove('/home/zzq/kaggle/house/feartures/feature_238.csv')

for i in range(30471):
    out=file('/home/zzq/kaggle/house/feartures/feature_238.csv','a+')
    xian=''
    for j in range(237):
        xian+=('%.6f'%feature_nd[i][j]+',')
    xian+=('%.6f'%feature_nd[i][237]+'\n')
    out.write(xian)
    out.close()

In [83]:
print feature_nd[3][3]

0.0514824186876


In [80]:
print type(feature_nd)
print type(feature_nd[4][5])

<type 'numpy.ndarray'>
<type 'float'>


In [75]:
all_lines=open('/home/zzq/kaggle/house/label_cls20').readlines()
y_label=[]
for i in range(0,len(all_lines)):
    lin=all_lines[i].strip('\n')
    data=int(lin)
    if data>=20:
        data=20
    y_label.append(data)

In [65]:
#what I need is feature_nd and y_label
writer = tf.python_io.TFRecordWriter('/home/zzq/kaggle/house/feartures/feature_238.tfrecord')
feature_raw = feature_nd.tostring()
example = tf.train.Example(
        features = tf.train.Features(
            feature = {'label':tf.train.Feature(int64_list = tf.train.Int64List(value = y)),
                       'feature':tf.train.Feature(bytes_list = tf.train.BytesList(value = [feature_raw]))}))
serialized = example.SerializeToString()
writer.write(serialized)
writer.close()

In [76]:
print len(y_label)

30471


In [66]:
#读取队列的数据
def read_and_decode(filename_queue):
    reader = tf.TFRecordReader()

    _, serialized_example = reader.read(filename_queue)

    features = tf.parse_single_example(
        serialized_example,
        # Defaults are not specified since both keys are required.
        features={
            'label': tf.FixedLenFeature([], tf.int64),
            'feature': tf.FixedLenFeature([], tf.string)
        })
    label=features['label']
    feature=features['feature']
    feature_raw=tf.decode_raw(feature,tf.float64)
    return feature_raw,label
filename_queue = tf.train.string_input_producer(
    ['/home/zzq/kaggle/house/feartures/feature_238.tfrecord'], num_epochs=10)
feature, label = read_and_decode(filename_queue)

In [96]:
one_hot=np.zeros((30471,20))
for i in range(30471):
    one_hot[i][y_label[i]-1]=1.0

In [ ]:
import tensorflow as tf

# Parameters
learning_rate = 0.001
training_epochs = 500
batch_size = 100
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 238 # MNIST data input (img shape: 28*28)
n_classes = 20 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])


# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
# with tf.device('/gpu:0'):
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    for epoch in range(training_epochs):
        sess.run(init)
        avg_cost = 0.
        # with tf.device('/gpu:0'):
        _, c = sess.run([optimizer, cost], feed_dict={x: feature_nd,
                                                          y: one_hot})
            # Compute average loss
        avg_cost += c / total_batch
    # Training cycle
    
        # 
        # total_batch = int(30471/batch_size)
        # # Loop over all batches
        # for i in range(total_batch):
        # 
        #     # Run optimization op (backprop) and cost op (to get loss value)
        #     
        # # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

In [105]:
with open('/home/zzq/kaggle/house/feartures/one_hot.csv','w') as my_file:
    np.savetxt(my_file,one_hot,delimiter=',')